In [31]:
import numpy as np
import polars as pl
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("darkgrid")

In [5]:
PATH_EXPERIMENT = Path("/beegfs/ws/0/s4610340-sleep_states/kaggle-detect_sleep_states/outputs/train/CNNSpectrogram-UNet1DDecoder/2023-11-23_16-08-35/single")
PATH_EXPERIMENT

PosixPath('/beegfs/ws/0/s4610340-sleep_states/kaggle-detect_sleep_states/outputs/train/CNNSpectrogram-UNet1DDecoder/2023-11-23_16-08-35/single')

In [12]:
predictions = np.load(PATH_EXPERIMENT / "preds.npy")
predictions.shape

(4814, 5760, 2)

In [11]:
labels = np.load(PATH_EXPERIMENT / "labels.npy")
labels.shape

(4814, 5760, 2)

In [30]:
keys = np.load(PATH_EXPERIMENT / "keys.npy")
keys.shape

(4814,)

In [27]:
import detect_sleep_states.config
import detect_sleep_states.data_module
import hydra


with hydra.initialize_config_dir(version_base=None, config_dir=str(PATH_EXPERIMENT / ".hydra")):
    cfg: detect_sleep_states.config.TrainConfig = hydra.compose(config_name="config")

features = detect_sleep_states.data_module.load_chunk_features(
    duration=cfg.duration,
    feature_names=cfg.features,
    series_ids=cfg.split.valid_series_ids,
    processed_dir=Path(cfg.dir.processed_dir),
    phase="train"
)

features

{'03d92c9f6f8a_0000000': array([[ 1.34292948e+00,  3.82847756e-01, -8.66025404e-01,
         -5.00000000e-01],
        [ 1.07496786e+00,  3.32763731e-01, -8.66025404e-01,
         -5.00000000e-01],
        [ 1.29597807e+00,  1.35310185e+00, -8.66025404e-01,
         -5.00000000e-01],
        ...,
        [ 4.99049544e-01, -3.94926786e-01, -2.58819045e-01,
          9.65925826e-01],
        [ 4.13803458e-01, -3.94926786e-01, -2.58819045e-01,
          9.65925826e-01],
        [ 6.49629452e-04, -3.85106415e-01, -2.58819045e-01,
          9.65925826e-01]]),
 '03d92c9f6f8a_0000001': array([[-0.04361605, -0.38805252,  0.        ,  1.        ],
        [ 0.21105517, -0.4057292 ,  0.        ,  1.        ],
        [-0.0559127 , -0.39885494,  0.        ,  1.        ],
        ...,
        [ 0.57469869, -0.4057292 ,  0.96592583, -0.25881905],
        [ 0.57469869, -0.4057292 ,  0.96592583, -0.25881905],
        [ 0.57469869, -0.4057292 ,  0.96592583, -0.25881905]]),
 '03d92c9f6f8a_0000002': arr

In [28]:
features["03d92c9f6f8a_0000000"].shape

(5760, 4)

In [29]:
cfg.features

['anglez', 'enmo', 'hour_sin', 'hour_cos']

In [ ]:
import matplotlib.patheffects as pe

sample_size: int = 5

fig: plt.Figure
fig, axs = plt.subplots(nrows=sample_size, ncols=1, figsize=(10, sample_size*4.5))

ax: plt.Axes

colors = sns.color_palette()

for ax, key_i in zip(axs, np.random.choice(len(keys), size=sample_size, replace=False)):
    chunk_features = features[keys[key_i]]
    
    anglez = chunk_features[:, 0]
    enmo = chunk_features[:, 1]
    
    ax.plot(enmo, label="enmo", color=colors[0], alpha=0.6)
    
    #ax.set_xticklabels([i * 5 / 3600 for i in range(len(enmo))])
    
    ax_anglez: plt.Axes = ax.twinx()
    ax_anglez.plot(anglez, label="anglez", color=colors[1], alpha=0.6)
    ax_anglez.set_yticks([])
    
    ax_prob: plt.Axes = ax.twinx()
    ax_prob.plot(predictions[key_i, :, 0], label="predictions[onset]", color=colors[2], linewidth=3, alpha=0.8)
    ax_prob.plot(predictions[key_i, :, 1], label="predictions[awake]", color=colors[3], linewidth=3, alpha=0.8)
    ax_prob.plot(labels[key_i, :, 0], label="labels[onset]", color=colors[2], linestyle="--", linewidth=3,
                 path_effects=[pe.Stroke(linewidth=3, foreground='white'), pe.Normal()])
    ax_prob.plot(labels[key_i, :, 1], label="labels[awake]", color=colors[3], linestyle=":", linewidth=1)
    
    ax_prob.legend(loc="upper left")
    ax_anglez.legend(loc="upper center")
    ax.legend(loc="upper right")
    
    